**Requirements**:

* QMCPy: `pip install qmcpy==2.1`
* LaTeX: `sudo apt update && sudo apt install -y texlive-full`
* testbook : `pip install testbook==0.4.2`
* Parsl: `pip install parsl==2025.7.28`

This notebook can be run interactively or in command line mode. To run in command line mode, use:
```bash
    jupyter nbconvert --to notebook --execute demos/talk_paper_demos/parsel_fest_2025/parsl_fest_2025.ipynb \
  --ExecutePreprocessor.kernel_name=qmcpy --ExecutePreprocessor.timeout=3600 --inplace
```

Our presentation slides for ParslFest are available at [Figma](https://www.figma.com/slides/k7EUosssNluMihkYTLuh1F/Parsl-Testbook-Speedup?node-id=174-95&t=t3jENVMltXWwdLdb-0).

In [5]:
try:
    import parsl as pl
except ModuleNotFoundError:
    !pip install -q parsl

In [6]:
import sys
import os
import time

# Ensure the path to the booktests directory is included (robust finder)
def _find_repo_root(start=os.getcwd()):
    cur = start
    while True:
        if os.path.exists(os.path.join(cur, 'pyproject.toml')):
            return cur
        parent = os.path.dirname(cur)
        if parent == cur:
            raise FileNotFoundError('repo root not found')
        cur = parent

sys.path.append(os.path.join(_find_repo_root(), 'test', 'booktests'))

# Configuration flags
# Set force_compute=False to reuse existing outputs; set True to force re-run
force_compute = True
is_debug = False

# Create output directory if it doesn't exist
print(f"{os.getcwd() = }")
output_dir = "output"
os.makedirs(output_dir, exist_ok=True)

# Clean local only files if force_compute is set
if force_compute:
    !(cd ../../.. && make clean_local_only_files)
    # remove output directory contents
    !rm -fr output/*.* runinfo/

os.getcwd() = '/workspaces/QMCSoftware/demos/talk_paper_demos/parsel_fest_2025'


## 1. Sequential Execution

In [13]:
seq_fname = os.path.join(output_dir, "sequential_time.csv")
if (not os.path.exists(seq_fname)) or force_compute:
    start_time = time.time()
    if is_debug:
        !(cd ../../.. && make booktests_no_docker TESTS="tb_quickstart tb_qmcpy_intro tb_quickstart tb_qmcpy_intro")
    else:
        !(cd ../../.. && make booktests_no_docker )
    sequential_time = time.time() - start_time
    
    # save sequential time to a file in output_dir
    with open(seq_fname, "w") as f:
        f.write(f"{sequential_time:.2f}\n")

Checking notebook ↔ booktest coverage...
    Skipping demos/DAKOTA_Genz/dakota_genz.ipynb (requires large manual file / heavy memory use)


    Skipping demos/talk_paper_demos/Argonne_Talk_2023_May/Argonne_2023_Talk_Figures.ipynb (heavy LaTeX + many figures; skipped in booktests_no_docker)
    Skipping demos/talk_paper_demos/MCQMC2022_Article_Figures/MCQMC2022_Article_Figures.ipynb (MCQMC 2022 article figures; not run in CI)
    Skipping demos/talk_paper_demos/ProbFailureSorokinRao/prob_failure_gp_ci.ipynb (prob_failure_gp_ci talk demo; heavy GP / prob. failure example)
    Skipping demos/talk_paper_demos/Purdue_Talk_2023_March/Purdue_Talk_Figures.ipynb (Purdue 2023 talk figures; not a CI booktest target)
    Skipping demos/talk_paper_demos/parsel_fest_2025/01_sequential.ipynb (helper notebook for parsl_fest_2025; not a standalone booktest)
    Skipping demos/talk_paper_demos/parsel_fest_2025/02_parallel.ipynb (helper notebook for parsl_fest_2025; not a standalone booktest)
    Skipping demos/talk_paper_demos/parsel_fest_2025/03_visualize_speedup.ipynb (helper notebook for parsl_fest_2025; not a standalone booktest)
    Sk

In [14]:
# Save actual timings per notebook in a file
import os
import pandas as pd

# Same output dir you already use
output_dir = "output"
os.makedirs(output_dir, exist_ok=True)

# Hand-entered from the sequential run you pasted
sequential_rows = [
    # order, test_name,                                  module,                                     mem_GB,  time_s
    ( 1, "test_mcqmc_2020_qmc_software_tutorial_notebook", "tb_MCQMC_2020_QMC_Software_Tutorial",   0.16,  16.70),
    ( 2, "test_acm_toms_sorokin_2025_notebook",            "tb_acm_toms_sorokin_2025",              0.16,   1.55),
    ( 3, "test_asian_option_mlqmc_notebook",               "tb_asian_option_mlqmc",                  0.16,  81.18),
    ( 4, "test_control_variates_notebook",                 "tb_control_variates",                    0.16,  10.65),
    ( 5, "test_digital_net_b2_notebook",                   "tb_digital_net_b2",                      0.16,  41.32),
    ( 6, "test_elliptic_pde_notebook",                     "tb_elliptic_pde",                        0.16,  69.91),
    ( 7, "test_gaussian_diagnostics_demo_notebook",        "tb_gaussian_diagnostics_demo",           0.16,  35.88),
    ( 8, "test_gbm_demo_notebook",                         "tb_gbm_demo",                            0.16,   1.59),
    ( 9, "test_iris_notebook",                             "tb_iris",                                0.16, 137.85),
    (10, "test_joss2025_notebook",                         "tb_joss2025",                            0.16, 386.16),
    (11, "test_lattice_random_generator_notebook",         "tb_lattice_random_generator",            0.16,  18.42),
    (12, "test_lebesgue_integration_notebook",             "tb_lebesgue_integration",                0.16,   3.82),
    (13, "test_linear_scrambled_halton_notebook",          "tb_linear_scrambled_halton",             0.16,  18.04),
    (14, "test_nei_demo_notebook",                         "tb_nei_demo",                            0.16,   7.89),
    (15, "test_plot_proj_function_notebook",               "tb_plot_proj_function",                  0.16,  14.21),
    (16, "test_pricing_options_notebook",                  "tb_pricing_options",                     0.16,  10.40),
    (17, "test_qei_demo_for_blog_notebook",                "tb_qei_demo_for_blog",                   0.16,  89.35),
    (18, "test_qmcpy_intro_notebook",                      "tb_qmcpy_intro",                         0.16,   4.12),
    (19, "test_quickstart_notebook",                       "tb_quickstart",                          0.16,   3.73),
    (20, "test_ray_tracing_notebook",                      "tb_ray_tracing",                         0.16,  37.62),
    (21, "test_sample_scatter_plots_notebook",             "tb_sample_scatter_plots",                0.16,   8.05),
    (22, "test_some_true_measures_notebook",               "tb_some_true_measures",                  0.16,   7.14),
    (23, "test_umbridge_notebook",                         "tb_umbridge",                            0.16,   1.11),
    (24, "test_vectorized_qmc_notebook",                   "tb_vectorized_qmc",                      0.16,  17.38),
    (25, "test_vectorized_qmc_bayes_notebook",             "tb_vectorized_qmc_bayes",                0.16,  70.94),
    (26, "test_why_add_q_to_mc_blog_notebook",             "tb_why_add_q_to_mc_blog",                0.16,   4.80),
]

seq_detail_df = pd.DataFrame(
    sequential_rows,
    columns=["order", "test_name", "module", "memory_gb", "time_s"],
)

# Sort by time if you want; otherwise keep original order
seq_detail_df = seq_detail_df.sort_values("order").reset_index(drop=True)

display(seq_detail_df)

detail_path = os.path.join(output_dir, "sequential_times_detail.csv")
seq_detail_df.to_csv(detail_path, index=False)
print(f"Saved detailed sequential timings to {detail_path}")

,order,test_name,module,memory_gb,time_s
0,1,test_mcqmc_2020_qmc_software_tutorial_notebook,tb_MCQMC_2020_QMC_Software_Tutorial,0.16,16.70
1,2,test_acm_toms_sorokin_2025_notebook,tb_acm_toms_sorokin_2025,0.16,1.55
2,3,test_asian_option_mlqmc_notebook,tb_asian_option_mlqmc,0.16,81.18
3,4,test_control_variates_notebook,tb_control_variates,0.16,10.65
4,5,test_digital_net_b2_notebook,tb_digital_net_b2,0.16,41.32
5,6,test_elliptic_pde_notebook,tb_elliptic_pde,0.16,69.91
6,7,test_gaussian_diagnostics_demo_notebook,tb_gaussian_diagnostics_demo,0.16,35.88
7,8,test_gbm_demo_notebook,tb_gbm_demo,0.16,1.59
8,9,test_iris_notebook,tb_iris,0.16,137.85
9,10,test_joss2025_notebook,tb_joss2025,0.16,386.16


Saved detailed sequential timings to output/sequential_times_detail.csv


In [15]:
# free memory
import gc
gc.collect();